In [1]:
import pathlib
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import load_model
from keras.callbacks import CSVLogger
import tensorflow_datasets as tfds
import numpy as np
import pandas as pd

In [2]:
# get current working directory
cwd = pathlib.Path.cwd()

In [3]:
ds1, ds2, ds3 = tfds.load('patch_camelyon', split=['train[:30%]', 'test[:7%]', 'validation[:7%]'], 
                                data_dir=cwd, 
                                download=False, 
                                shuffle_files=True)

In [4]:
# Function to preprocess the images and labels
def preprocess(data):
    image = data['image']
    label = data['label']
    image = tf.image.resize(image, [224, 224])  # Resize to 224x224
    image = preprocess_input(image)  # Apply ResNet-specific preprocessing
    return image, label

In [5]:
# Apply the preprocessing function to the dataset
transfer_train_data = ds1.map(preprocess).batch(32)
transfer_validation_data = ds2.map(preprocess).batch(32)
transfer_test_data = ds3.map(preprocess).batch(32)

In [6]:
resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [7]:
# Freeze the base model layers
for layer in resnet_model.layers:
    layer.trainable = False

In [8]:
# Transfer Model Architecture
transfer_model = tf.keras.Sequential([
    resnet_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [9]:
transfer_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Create CSVLogger callback to save history
csv_filepath = 'ae_low_training_log.csv'
csv_logger = CSVLogger(csv_filepath, append=True)

history = transfer_model.fit(
    transfer_train_data,
    validation_data=transfer_validation_data,
    epochs=50,
    callbacks=[csv_logger]  # Add custom callback here
)

Epoch 1/50
  91/2458 [>.............................] - ETA: 12:13 - loss: 0.5393 - accuracy: 0.7995

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 5))

# Plot Training and Validation Loss
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training loss')
plt.plot(history.history['val_loss'], label='Validation loss')
plt.title('Training and Validation Loss')
plt.legend()

# Plot Training and Validation Accuracy
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training accuracy')
plt.plot(history.history['val_accuracy'], label='Validation accuracy')
plt.title('Training and Validation Accuracy')
plt.savefig('transfer_learning_loss_acc.png')
plt.legend()

plt.show()

In [ ]:
transfer_model.save('transfer_model.h5')

In [ ]:
model = load_model('transfer_model.h5') 

# Predict on test data
predictions = model.predict(transfer_test_data)

predicted_labels = np.argmax(predictions, axis=1)

predictions_pd = pd.DataFrame({
    'Id': list(range(len(predictions))),
    'Predicted': predictions.reshape(-1,),
})

predictions_pd.to_csv('final_predictions_transfer_learning.csv', index=False)

In [ ]:
true_labels = []
for images, labels in transfer_test_data.unbatch().take(len(predictions)):
    true_labels.append(labels.numpy())

# Convert the true labels to a numpy array
true_labels = np.array(true_labels)

# If true_labels is one-hot encoded (2D), convert it to class indices
if true_labels.ndim == 2:
    true_labels = np.argmax(true_labels, axis=1)

# Calculate the accuracy
accuracy = np.mean(predicted_labels == true_labels)
my_string = f"Accuracy: {accuracy * 100:.2f}%"
print(my_string)

# Write the accuracy to a csv file
with open("accuracy_transfer_learning.csv", 'w') as file:
    file.write(my_string)
